# Modules

In [3]:
import pandas as pd
import numpy as np
import unidecode
from difflib import SequenceMatcher
import salem
import matplotlib.pyplot as plt 
import re
import os
% matplotlib inline

# File paths

In [4]:
f_path = 'C:\\Users\\jlandman\\Desktop\\database_Fischer_et_al._2015_The_Cryosphere.txt'                 # Fischer database with swiss coordinates 
fll_path = 'C:\\Users\\jlandman\\Desktop\\SGI2010wgs84_shapefiles\\parameters_SGI2010.csv'               # Fischer database with lat/lon
a_path = 'C:\\Users\\jlandman\\Desktop\\DOI-WGMS-FoG-2015-11\\WGMS-FoG-2015-11-A-GENERAL-INFORMATION.csv'# FoG: A GENERAL
b_path = 'C:\\Users\\jlandman\\Desktop\\DOI-WGMS-FoG-2015-11\\WGMS-FoG-2015-11-B-STATE.csv'              # FoG: B STATE
d_path = 'C:\\Users\\jlandman\\Desktop\\DOI-WGMS-FoG-2015-11\\WGMS-FoG-2015-11-D-CHANGE.csv'             # FoG: D CHANGE

# Read files

In [5]:
pdf = pd.read_csv(f_path, sep = '\t', encoding='iso-8859-1')
pdfll = pd.read_csv(fll_path, sep= ';', encoding='iso-8859-1', usecols=[2,3,6,14,15]) # we only need ID and x/y here
pda = pd.read_csv(a_path, encoding='iso-8859-1')
pdb = pd.read_csv(b_path, encoding='iso-8859-1')
pdd = pd.read_csv(d_path, encoding='iso-8859-1')

# Preselect FoG IDs in Switzerland

In [6]:
pda = pda[pda.POLITICAL_UNIT == 'CH']
pdb = pdb[pdb.WGMS_ID.isin(pda.WGMS_ID.values)]
pdd = pdd[pdd.WGMS_ID.isin(pdd.WGMS_ID.values)]

# Haversine function

In [7]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between one point 
    on the earth and an array of points (specified in decimal degrees)
    """
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371000 # Radius of earth in meters
    return c * r

# Some settings

In [8]:
begin_new_id = 7000  # where to start with assigning new WGMS IDs
new_id_range = range(begin_new_id, begin_new_id+len(pdf)+1,1)
new_id_range

range(7000, 8386)

# Correct missing underscores and column names

In [9]:
pdf = pdf.rename(columns={'uncertainty_dvol_between_t1_and_t2_mio m3': 'uncertainty_dvol_between_t1_and_t2_mio_m3'})
pdfll = pdfll.rename(columns={'uncertainty_dvol_between_t1_and_t2_mio m3': 'uncertainty_dvol_between_t1_and_t2_mio_m3'})

pdfll = pdfll.rename(columns={'Unnamed: 15': 'Glacier_name_SGI2010'})

# Find matching glaciers based on FoG => compare names, area and location

### Dictionary of strings and symbols that need to be replaced prior to calculation of similarity score

In [10]:
name_adjust = {'gletscher':'g', 'glacier':'g' , 'vadret':'v', 'ghiacciaio':'g', 'vedretta':'v', 'ferner':'f', 'ä':'ae', 'ö':'oe', 'ü':'ue', 'é':'e', 'à':'a', 'è':'e', 'ô':'o', 'ß':'ss'} 

### Introduce FoG column with new names

In [11]:
pda['FoG_compname'] = ''

for initname in [n for n in pda.NAME.values]:
    name = initname.lower()
    for key in name_adjust:
        if key in name:
            name = name.replace(key, name_adjust[key])
    #pda.FoG_compname[pda.NAME == initname] = name
    pda.loc[pda.NAME == initname, 'FoG_compname'] = name

### Introduce some new columns

In [12]:
pdf.COMPNAME = ''              # simplified name 
pda.MATCH_RATIO = np.nan       # string matching ratio 
pda.MATCH_NAME = ''            # Name of the FoG glacier that matches Mauro's name best
pda.CLOSEST = ''               # closest FoG glacier point
pda.DIST_TO_CLOSEST = np.nan   # distance of Mauro's point to the closest FoG point 
pda.AREA_CLOSEST = np.nan      # column with the area of Mauro's glacier found by string matching 
pda.AREA_MATCH = np.nan        # column with the area of Mauro's glacier found by string matching
pda.AREA = np.nan

### Adjust also Mauro's names to make them comparable

In [13]:
for idx,cols in pdf.iterrows():
    # simplify name
    compname_mau = cols.Glacier_name_SGI2010.lower()
    for key in name_adjust:
        if key in compname_mau:
            # replace the umlauts etc.
            compname_mau = compname_mau.replace(key, name_adjust[key])
    
    # delete the bracket stuff in order to improve the ratio
    start = compname_mau.find('(')
    if start != -1:
        compname_mau = compname_mau[0:start]
    compname_mau = compname_mau.replace('*', '')
    compname_mau = compname_mau.strip()

    pdf.loc[pdf.Glacier_name_SGI2010 == cols.Glacier_name_SGI2010, 'COMPNAME'] = compname_mau

### Find matching glaciers for the 159 swiss glaciers in PDA

for fidx,fcols in pda[0:10].iterrows():
    
    # create an AREA column entry (from the "state" table)
    try: # take the latest area entry
        area_match = pdb[pdb.WGMS_ID == fcols.WGMS_ID].AREA.values[~np.isnan(pdb[pdb.WGMS_ID == fcols.WGMS_ID].AREA.values)][-1]
    except IndexError:
        area_match = np.nan
    pda.loc[pda.WGMS_ID == fcols.WGMS_ID, 'AREA'] = area_match
    
    
    # find biggest ratio of string matching and insert
    ratio = np.nan
    name = ''
    for cname in pdf['COMPNAME'].values:
        
        curr_ratio = SequenceMatcher(None, fcols.FoG_compname, cname).ratio()
        
        if curr_ratio > ratio or pd.isnull(ratio):  #the latter in order to catch the initial case
            ratio = curr_ratio
            name = cname
            
    pda.loc[pda.NAME == fcols.NAME, 'MATCH_RATIO'] = ratio
    pda.loc[pda.NAME == fcols.NAME, 'MATCH_NAME'] = name
    
    # insert the area (at t2, because this is the latest) of the glacier found by string matching
    pda.loc[pda.NAME == fcols.NAME, 'AREA_MATCH'] = pdf[pdf['COMPNAME'] == name].area_t2_km2.iloc[0]
    
    
    #find closest pdf glacier
    dist = np.nan
    close_name = ''
    for pdf_idx, pdf_cols in pdf.iterrows():
        lat = pdfll[pdfll.Glacier_name_SGI2010 == pdf_cols.Code_SGI2010]['y WGS84)'].values[0]
        lon = pdfll[pdfll.Glacier_name_SGI2010 == pdf_cols.Code_SGI2010]['Location(x WGS84)'].values[0]
        curr_dist = haversine(lon, lat, fcols.LONGITUDE, fcols.LATITUDE)
        
        if curr_dist < dist or pd.isnull(dist): # the second is for the initial loop
            dist = curr_dist
            close_name = pdf_cols.COMPNAME
    
    pda.loc[pda.NAME == fcols.NAME, 'DIST_TO_CLOSEST'] = dist
    pda.loc[pda.NAME == fcols.NAME, 'CLOSEST'] = close_name
    
    print(fcols.NAME)
    # insert the area (at t2, because this is the latest) of the glacier ehic is found to be the closest
    pda.loc[pda.NAME == fcols.NAME, 'AREA_CLOSEST'] = pdf[pdf['COMPNAME'] == close_name].area_t2_km2.iloc[0]

In [14]:
pdfll.COMPNAME = ''              # simplified name 

In [15]:
for idx,cols in pdfll.iterrows():
    # simplify name
    compname_mau = cols.Names.lower()
    for key in name_adjust:
        if key in compname_mau:
            # replace the umlauts etc.
            compname_mau = compname_mau.replace(key, name_adjust[key])
    
    # delete the bracket stuff in order to improve the ratio
    start = compname_mau.find('(')
    if start != -1:
        compname_mau = compname_mau[0:start]
    compname_mau = compname_mau.replace('*', '')
    compname_mau = compname_mau.strip()

    pdfll.loc[pdfll.Names == cols.Names, 'COMPNAME'] = compname_mau
  

In [ ]:
#if os.path.exists('assigned_automated.csv'):
#    pass

#else:
for fidx,fcols in pda.iterrows():

    # create an AREA column entry (from the "state" table)
    area_match = np.nan
    try: # take the latest area entry
        area_match = pdb[pdb.WGMS_ID == fcols.WGMS_ID].AREA.values[~np.isnan(pdb[pdb.WGMS_ID == fcols.WGMS_ID].AREA.values)][-1]
    except IndexError:
        area_match = np.nan
    pda.loc[pda.WGMS_ID == fcols.WGMS_ID, 'AREA'] = area_match


    # find biggest ratio of string matching and insert
    ratio = 0.0
    name = ''
    for cname in pdfll['COMPNAME'].values:
        curr_ratio = SequenceMatcher(None, cname, fcols.FoG_compname).ratio()

        if curr_ratio > ratio or ratio==0.0:  #the latter in order to catch the initial case
            ratio = curr_ratio
            name = cname

    pda.loc[pda.NAME == fcols.NAME, 'MATCH_RATIO'] = ratio
    pda.loc[pda.NAME == fcols.NAME, 'MATCH_NAME'] = name

    # insert the area (at t2, because this is the latest) of the glacier found by string matching
    pda.loc[pda.NAME == fcols.NAME, 'AREA_MATCH'] = pdfll[pdfll['COMPNAME'] == name]['area(km2)'].iloc[0]


    #find closest pdf glacier
    dist = np.nan
    close_name = ''
    for pdf_idx, pdf_cols in pdfll.iterrows():
        lat = pdfll[pdfll.Names == pdf_cols.Names]['y WGS84)'].values[0]
        lon = pdfll[pdfll.Names == pdf_cols.Names]['Location(x WGS84)'].values[0]
        curr_dist = haversine(lon, lat, fcols.LONGITUDE, fcols.LATITUDE)

        if curr_dist < dist or pd.isnull(dist): # the second is for the initial loop
            dist = curr_dist
            close_name = pdf_cols.COMPNAME

    pda.loc[pda.NAME == fcols.NAME, 'DIST_TO_CLOSEST'] = dist
    pda.loc[pda.NAME == fcols.NAME, 'CLOSEST'] = close_name

    print(fcols.NAME)
    # insert the area (at t2, because this is the latest) of the glacier ehic is found to be the closest
    pda.loc[pda.NAME == fcols.NAME, 'AREA_CLOSEST'] = pdfll[pdfll['COMPNAME'] == close_name]['area(km2)'].iloc[0]


    pda.to_csv('assigned_automated.csv')

In [17]:
pda.to_csv('assigned_automated.csv')

In [290]:
pda[['NAME', 'FoG_compname', 'MATCH_RATIO', 'MATCH_NAME', 'DIST_TO_CLOSEST', 'CLOSEST', 'AREA', 'AREA_MATCH', 'AREA_CLOSEST']][100:110]

,NAME,FoG_compname,MATCH_RATIO,MATCH_NAME,DIST_TO_CLOSEST,CLOSEST,AREA,AREA_MATCH,AREA_CLOSEST
793,PLATTALVA,plattalva,0.600000,piz muttala,858.783359,griessfirn-n,0.73,0.06338,0.33594
794,PORCHABELLA,porchabella,0.814815,porchabella v da,230.998086,porchabella v da,2.59,1.67562,1.67562
795,PRAPIO,prapio,0.705882,prapio g du,507.398171,prapio g du,0.36,0.21156,0.21156
796,PUNTEGLIAS,punteglias,0.714286,fuorcla punteglias,455.219529,piz urlaun-s,0.93,0.25875,0.22156
797,RAETZLI (PLAINE MORTE),raetzli (plaine morte),0.619048,g de la plaine morte,1242.784421,g de la plaine morte,9.80,7.29375,7.29375
798,RHONE,rhone,0.909091,rhoneg,218.315423,rhoneg,15.80,15.31000,15.31000
799,RIED,ried,0.888889,riedg,1359.600643,duerreng,8.26,7.31687,0.08040
800,ROSEG,roseg,0.727273,rhoneg,1329.859833,chapuetschin v dal,8.72,15.31000,0.34719
801,ROSENLAUI,rosenlaui,0.947368,rosenlauig,779.049789,rosenlauig,5.90,5.40250,5.40250
802,ROSSBODEN,rossboden,0.888889,rossbodeg,296.848823,senggchuppa,1.89,1.18125,0.05822


In [331]:
i = 'SEEWJINEN'

In [332]:
ID = pda[pda.NAME == i].WGMS_ID.iloc[0]

In [334]:
pda[pda.NAME == i]#[['LATITUDE', 'LONGITUDE']]

,POLITICAL_UNIT,NAME,WGMS_ID,RIVER_BASIN,FREE_POSITION,LOCAL_CODE,LOCAL_PSFG,GEN_LOCATION,SPEC_LOCATION,LATITUDE,...,GEO-REGION_CODE,GEO-SUBREGION_CODE,FoG_compname,AREA,MATCH_RATIO,MATCH_NAME,AREA_MATCH,DIST_TO_CLOSEST,CLOSEST,AREA_CLOSEST
813,CH,SEEWJINEN,3333,NaN,NaN,NaN,NaN,NaN,NaN,46.0,...,CEU,CEU-01,seewjinen,NaN,0.9,seewjinen g,1.42375,664.159293,seewjinen g,1.42375


In [325]:
pdfll[pdfll['COMPNAME'] == pda[pda.NAME==i].MATCH_NAME.iloc[0]]

,Location(x WGS84),y WGS84),area(km2),Names,Glacier_name_SGI2010,COMPNAME
320,8.42756,46.651699,0.83609,ROTFIRN,A51F/68n,rotfirn


In [326]:
pdb[pdb.where(~pd.isnull(pdb.AREA)).WGMS_ID == ID].tail(10)#[['NAME', 'LENGTH', 'AREA']]

,POLITICAL_UNIT,NAME,WGMS_ID,YEAR,HIGHEST_ELEVATION,MEDIAN_ELEVATION,LOWEST_ELEVATION,ELEVATION_UNC,LENGTH,LENGTH_UNC,AREA,AREA_UNC,SURVEY_DATE,SURVEY_PLATFORM_METHOD,PUB_IN_FOG,INVESTIGATOR,SPONS_AGENCY,REFERENCE,REMARKS
2566,CH,ROTFIRN NORD,430,1973,NaN,2680.0,NaN,NaN,2.3,NaN,1.21,NaN,19739999.0,NaN,1975.0,NaN,NaN,NaN,NaN
2567,CH,ROTFIRN NORD,430,1975,3525.0,2680.0,2020.0,NaN,2.3,NaN,1.21,NaN,19759999.0,NaN,1975.0,NaN,NaN,NaN,NaN


In [335]:
pdfll[pdfll['Names'].str.contains('seew', case=False)]

,Location(x WGS84),y WGS84),area(km2),Names,Glacier_name_SGI2010,COMPNAME
914,7.9485,46.0009,1.42375,SEEWJINEN GLETSCHER,B52/22,seewjinen g


## A dictionary with links from FoG to Mauro's database (full names in FoG to full and short names in Mauro's DB)

In [301]:
links = {
    'A NEUVE GL. L\'':('','','',False),   # no equivalent for whole glacier: A NEUVE GLACIER DE L'-S (B85/07) and A NEUVE GLACIER DE L'-N (B85/08)
    'ADLER':('ADLERGLETSCHER (Teilgl. von B56/03)','adlerg','B56/14n', True),  # no area in FoG, but quite obvious
    'ALBIGNA':('ALBIGNA VADREC D\' (Nr. 116)','albigna vadrec d\'', 'C84/16', True),  # ok, error in area in pdb  
    'ALLALIN':('Allalingletscher* (Teilgl. von B52/66n)','allaling', 'B52/29', True), #ok area 9.68/9.17
    'ALPETLI(KANDER)':('KANDERFIRN (Teilgl. von A55B/29n; Nr. 109)','kanderfirn','A55B/13',True),  # ok area 14/12
    'ALTELS':('','','',False),     # no equivalent for whole glacier. Altels-S A55C/04, Altels-NW A55C/03 and Altels-SE A55C/18n
    'AMMERTEN':('','','',False),   # no equivalent for whole glacier. Ammerten-W A55F/07n and AMMERTEN-E (Nr.111) A55F/01
    'AROLLA (BAS)':('','',False),  # no equivalent in Mauro's DB (is included in Glacier du Mont Collon B73/14)
    'BALMHORN':('BALMHORNGLETSCHER (Teilgl. von A55B/42n)','balmhorng','A55B/18',True),  # ok (area: 1.7/1.9)
    'BASODINO':('','','',False),  # area bigger in Mauro's DB, even for only BASODINO GH. DEL (Nr. 104) (1.84/1.89). What to do with basodino-N and basodino-NW?
    'BIDER':('BIDERGLETSCHER','biderg','B53/08',True),  # ok (no area in FoG, but unique)
    'BIFERTEN':('BIFERTENFIRN (Nr. 77)','bifertenfirn','A50I/12',True),  # ok (area: 2.86/2.5)
    'BIRCH':('BIRCHGLETSCHER','birchg','B32/06',True),   # ok even though area 0.54/0.22
    'BIS':('BISGLETSCHER (Nr. 107)','bisg','B58/08',True),    # ok even though area 4.79/3.82
    'BLUEMLISALP':('BLÜMLISALPGLETSCHER (Nr. 64)','bluemlisalpg','A55B/02',True), # ok due to lat/lon area 2.22/2.98
    'BODMER':('BODMER','bodmerg','C02/02',True),   # Link should be okay but area 0.64/0.32
    'BOVEYRE':('BOVEIRE GLACIER DE (Nr. 41)','boveire g de','B84/04',True),  # ok (area 1.99/1.62)
    'BREITHORN':('','','',False),   # unclear...many entries in Mauro's DB, but no values in FoG => can be easily implemented in FoG without caring about ID
    'BRENEY':('BRENAY GLACIER DU (Nr. 36)','brenay g du','B82/19',True),   # ok (areas 9.8/7.1)
    'BRUNEGG':('BRUNEGGGLETSCHER (Teilgl. von B60/09; Nr. 20)','bruneggg','B60/20n',True),  # ok areas 6.1/5.5
    'BRUNNI':('BRUNNIFIRN (Nr. 72)','brunnifirn','A51D/15',True),  # ok areas 2.99/2.31
    'CALDERAS':('CALDERAS VADRET (Nr. 95)','calderas v','E35/17',True),   # ok even though areas 1.2/0.66
    'CAMBRENA':('','','',False),  # is Cambrena Vadret dal (Teilgl. von C93/09) C93/11n, but what about Cambrena-E* (Teilgl. von C93/09) C93/08?  
    'CAVAGNOLI':('CAVAGNÖÖ GH. DEL (Nr. 119)','cavagnoeoe gh. del','C14/17',True), # ok due to lat/lon area 1.32/0.44 (ok as compared with SGI1973)
    'CHEILLON':('CHEILON GLACIER DE (Nr. 29)','cheilon g de','B74/08',True), # okay even though area 4.73/3.6. Spelling?
    'CLARIDENFIRN':('','','',False),  # unclear case => too many equivalent in Mauro's DB and no area/length in FoG, case for PARENT ID
    'CORBASSIERE':('Corbassière (Teilgl. von B83/03)*','corbassiere','B83/15n',True),  # ok. unclear if also Combin de Corbassière-E (Teilgl. von B83/03)* is meant (area: 0.4km2), but separated by ridge
    'CORNO':('CORNO GH. DEL (Nr. 120)','corno gh. del','C34/01',True),  # ok even though area 0.27/0.1
    'DAMMA':('DAMMAGLETSCHER (Nr. 70)','dammag','A51F/10',True),  # ok even though area 6.32/4.24
    'DOLENT GL. DU':('DOLENT GLACIER DU','dolent g du','B85/04',True),  # ok even though noe area given in FoG
    'DUNGEL':('TUNGELGLETSCHER (Teilgl. von A56D/09n, Nr. 112)','tungelg','A56D/01',True),  #ok area 1.2/0.93
    'EIGER':('EIGERGLETSCHER (Nr. 59)','eigerg','A54M/03',True),   # ok area 2.27/1.53
    'EIGER (WEST)':('','','',False),  # unclear, possibly Eiger Südwestgrat* A54M/02
    'EN DARREY':('EN DARREY GLACIER DE L\' (Nr. 30)','en darrey g de l\'','B74/11',True), #ok (areas 1.86/1.28)
    'FEE NORTH':('','','',False),  # probably all of FEEGLETSCHER-S-I B53/14n, FEEGLETSCHER-S-II B53/15n, Feegletscher-N-I (Alphubel)* (Teilgl. von B53/...B53/17n, Feegletscher-N-I (Täschhorn)* (Teilgl. von B53...B53/18n, Feegletscher-N-I (Dom)* (Teilgl. von B53/16n) B53/19n, FEEGLETSCHER-N-II B53/20n, Feechopf-W* B55/17 
    'FERPECLE':('FERPÈCLE GLACIER DE (Nr. 25)','ferpecle g de','B72/11',True),  # ok areas 9.79/9.0
    'FIESCHER':('FIESCHERGLETSCHER VS (Teilgl. von B40/14n, Nr. 4)','fiescherg vs','B40/07',True), # ok area 33.06/29.48
    'FIRNALPELI':('','','',False),  # FoG lat/lon unclear;can be combinations of FIRNALPELIGLETSCHER-E (Nr. 75) A51H/13 and FIRNALPELIFIRN A51H/23n OR FIRNALPELIGLETSCHER-E (Nr. 75) and FIRNALPELIGLETSCHER-W
    'FLUCHTHORN GL.':('','','', False) # should be Fluchthorn-NE* B52/26 and maybe also Fluchthorn-E* B52/25 (area 0.37/(0.26+0.01))
    'FORNO':('FORNO VADREC DEL (Nr. 102)','forno vadrec del','C83/12',True), # ok area 8.7/5.3; NICHT Ofenhorn-W* (lat/lon!)
    'GAMCHI':('GAMCHIGLETSCHER (Nr. 61)','gamchig','A55A/04',True),  # ok area (1.73/1.23)
    'GAULI':('GAULIGLETSCHER (Teilgl. von A54I/19n) Nr. 52','gaulig','A54I/05',True),  # ok (area 13.7/11.4)
    'GELTEN':('','','',False),  # ok if GELTENGLETSCHER-E (Nr. 113) A56D/05 (area 1.17/0.81) but should be also GELTENGLETSCHER-W (area 0.45)
    'GIETRO':('GIETRO GLACIER DU (Nr. 37)','gietro g du','B82/14',True),  # ok (area 5.54/5.16)
    'GLAERNISCH':('GLÄRNISCHFIRN (Nr. 80)','glaernischfirn','A50K/04',True), # ok (area 2.09/1.41)
    'GORNER':('','','',False),  # problem: gorner not in Mauro's DB; Grenz is too big (whole glacier)
    'GRAND PLAN NEVE':('','','',False), # is plan neve-e (area 0.12/0.18) due to lat/lon of FoG point
    'GRIES':('GRIESGLETSCHER (Nr. 3)','griesg','B45/04',True),  # ok (area 4.83/4.78)
    'GRIESSEN(OBWA.)':('GRIESSENFIRN','GRIESSENFIRN','A51H/02',True),  # ok (area 1.27/0.86)
    'GRIESSERNU':('GRIESSERNU GLETSCHER','griessernu g','C02/06',True), # ok even though no area in FoG
    'GROSSER ALETSCH':('GROSSER ALETSCH GLETSCHER (Teilgl. von B36/49n) Nr. 5','grosser aletsch g','B36/26',True), # ok (area 81.3/78.3)
    'GRUBEN':('','','',False), # should be all GRÜEBUGLETSCHER-N-II (Teilgl. von B51/17n), GRÜEBUGLETSCHER-S (Teilgl. von B51/17n), GRÜEBUGLETSCHER-N-I (Teilgl. von B51/17n), NAME SHOULD  BE CHANGED IN FoG
    'GUTZ':('GUTZGLETSCHER','gutzg','A54L/02',True),  # ok even though no area in FoG
    'HANGENDE':('HANGENDE GLETSCHER','hangende g', 'B52/27',True), # ok due to lat/lon
    'HOHLAUB':('','','',False), # both Hohlaubgrat-E* (Teilgl. von B52/67n) and Hohlaub Gletscher* (Teilgl. von B52/67n)
    'HOMATTU':('','','',False), #  HOMATTUGLETSCHER-II and HOMATTUGLETSCHER-I
    'HUEFI':('HÜFIFIRN (Nr. 73)','huefifirn','A51D/10',True),  # ok (area 13.73/12.72)
    'KALTWASSER':('CHALTWASSERGLETSCHER (Nr. 7)','chaltwasserg','B47/04',True),  # ok (areas 18.5/1.49)
    'KEHLEN':('CHELENGLETSCHER (Totalgl.; Nr. 68)','cheleng','A51F/15',True),  # ok even though area 1.73/3.15
    'KESSJEN':('CHESSJENGLETSCHER-E (Nr. 12)','chessjeng-e','B52/33',True), #ok (areas 0.19/0.19) there is now chessjengl.-w!
    'LAEMMERN (WILDSTRUBEL)':('WILDSTRUBELGLETSCHER (Teilgl. von A55C/24n) Nr. 63','wildstrubelg','A55C/13',True), # ok even though area 3.15/2.34
    'LANG':('Langgletscher (Totalgl.; Nr. 18)','langg','B31/19n',True),  # ok areas 10.03/8.26
    'LAVAZ':('','','',False),  # LAVAZ GLATSCHER DA (Nr. 82) and Lavaz-W*
    'LIMMERN':('LIMMERNFIRN (Nr. 78)','limmernfirn','A50I/06',True), # ok (area 2.41/1.89)
    'LISCHANA':('','','',False),  # ok vadret da triazza is one of the two remnants (one is no longer mapped)
    'MAIGHELS EAST BRANCH':('MAIGHELS GLATSCHER DA-E','maighels glatscher da-e','A14I/04',True),  # ok but no area in FoG
    'MAIGHELS WEST BRANCH':('MAIGHELS GLATSCHER DA-W','maighels glatscher da-w','A14I/05',True),  # ok but no area in FoG
    'MARTINETS':('MARTINETS GLACIER DES (Nr. 46)','martinets g des','B17/08',True), # ok area 0.59/0.36
    'MINSTIGER':('MINSTIGERGLETSCHER','minstigerg','B41/07',True),  # ok areas 3.09/2.25
    'MITTELALETSCH':('MITTELALETSCHGLETSCHER (Teilgl. von B36/49n) Nr. 106','mittelaletschg','B36/21',True), # ok area 8.5/6.8
    'MOIRY':('MOIRY GLACIER DE (Nr. 24)','moiry g de','B64/02',True), # ok area 6.11/4.89
    'MOMING':('MOMING GLACIER DE (Nr. 23)','moming g de','B62/10',True),  # ok , maybe also Pointe Nord de Moming-SE* and Blanc de Moming-W*
    'MONT FORT (ZINAL)':('','','',False),  # not clear, probably PETIT M. FORT GLACIER DU (B75/07), but area is still too low
    'MONT MINE':('MONT MINÉ GLACIER DU (Nr. 26)','mont mine g du','B72/15',True), # ok areas 10.3/9.9
    'MONTO MORO GL.':('Monte Moro-W*','monte moro-w','B52/21',True),  # only a remnant obviously
    'MORTERATSCH':('MORTERATSCH VADRET DA (Totalgl.; Nr. 94)','morteratsch v da','E22/03',True), # ok areas 17/15
    'MURTEL':('MURTEL VADRET DAL','murtel v dal', 'E23/16',True),  # ok, NOT E24/04!!! 
    'MUTT':('MUTTGLETSCHER (Nr. 2)','muttg','B44/03',True), # ok, area 0.57/0.36
    'MUTTEN':('Muttengletscher* (Teilgl. von A51E/23)','mutteng','A51E/56n',True), #ok, no areas in FoG
    'OB.GRINDELWALD':('OBERER GRINDELWALDGLETSCHER (Nr. 57)','oberer grindelwaldg','A54L/04',True), # ok areas 10.07/8.41
    'OBERAAR':('OBERAARGLETSCHER (Teilgl. von A54G/35n) Nr. 50','oberaarg','A54G/03',True), # ok areas 5.23/4.10
    'OFENTAL':('OFENTAL GLETSCHER (Nr. 9)','ofental g','B52/17',True),  #ok even though areas 0.4/0.04...possibly one remnant missing in Mauro's DB (see swisstopo)
    'OTEMMA':('Otemma (Teilgl. von B82/27)*','otemma','B82/51n',True),  # ok areas 16.55/12.59
    'PALUE':('Palü Vadret da (Teilgl. von C93/04)','palue v da','C93/10n',True),  # ok areas 6.62/5.26
    'PANEYROSSE':('PANEIROSSE GLACIER DE (Nr. 44)','paneirosse g de','B17/02',True),  # ok areas 0.45/0.3
    'PARADIES':('PARADIESGLETSCHER (Nr. 86)','PARADIESGLETSCHER (Nr. 86)','A13N/06',True),  # ok 4.6/1.8
    'PARADISINO (CAMPO)':('','','',False),  # unclear, possibly CAMP VEDREIT DA (Nr. 101)
    'PIERREDAR':('PIERREDAR GLACIER DE (Nr. 49)','pierredar g de','B16/05',True), # ok areas 0.67/0.3
    'PIZOL':('PIZOLGLETSCHER (Nr. 81)','pizolg','A50D/01',True), # ok areas 0.32/0.08
    'PLAINE MORTE':('','','',False), # multiple: GLACIER DE LA PLAINE MORTE (Nr. 65), PLAINE MORTE-W GLACIER DE LA and PLAINE MORTE-E GLACIER DE LA, area in FoG missing!!!
    'PORCHABELLA':('PORCHABELLA VADRET DA (Nr. 88)','porchabella v da','A12E/04',True),  # ok area 2.59/1.67
    'PRAPIO':('PRAPIO GLACIER DU (Nr. 48)','prapio g du','B16/03',True),   # ok area 0.36/0.21
    'PUNTEGLIAS':('Bündner Tödi*','buendner toedi','A14M/08',True),  #  sounds strange but ok area 0.93/0.67
    'RAETZLI (PLAINE MORTE)':('','','',False),  # three polygons: GLACIER DE LA PLAINE MORTE (Nr. 65) A55F/03, PLAINE MORTE-W GLACIER DE LA B23/09n and PLAINE MORTE-E GLACIER DE LA B24/04n
    'RHONE':('Rhonegletscher* (Teilgl. von B43/03)','rhoneg','B43/12n',True),  # ok areas 15.8/15.31
    'RIED':('RIEDGLETSCHER (Nr. 17)','riedg','B54/03',True),  # ok areas 8.26/7.31
    'ROSEG':('ROSEG VADRET DA (Nr. 92)','roseg v da','E23/11',True),  # ok areas 8.71/6.81
    'ROSENLAUI':('ROSENLAUIGLETSCHER (Nr. 56)','rosenlauig','A54J/02',True),  # ok areas 5.9/5.4
    'ROSSBODEN':('ROSSBODEGLETSCHER (Nr. 105)','rossbodeng','C02/04',True),  # ok areas 1.89/1.18
    'ROTFIRN NORD':('Schattmigstock* (Rotfirn-N, Nr. 69)','schattmigstock','A51F/13',True)  #  ok area 1.21/0.91
    'ROTTAL':('ROTTALGLETSCHER-NW (Teilgl. von B52/02)','rottalg-nw','B52/37n',True),  #  ok no area in FoG
    'SALEINA':('','','',False),  # is Saleina* (Teilgl. von B85/16), but area is too high (6.54) compared to FoG (5.03)
    'SANKT ANNA':('ST. ANNAFIRN (Nr. 67)','st. annafirn','A51E/12',True),  # ok even though areas 0.41/0.22 (might be debris problem)
    'SARDONA':('','','',False),  # should be Sardonagletscher-II*, but area is too high (0.45, FoG only 0.38)
    'SCALETTA':('SCALETTAGLETSCHER (Nr. 115)','scalettag','A12D/03',True),  # ok even though area 0.66/0.21 (debris?)
    'SCHOENBUEHL GL.':('','','',False),  # is probably both SCHÖNBÜHLGLETSCHER-SE and SCHÖNBÜHLGLETSCHER-NW (no FoG area given)
    'SCHWARZ':('SCHWARZGLETSCHER (Nr. 62)','schwarzg','A55C/05',True),  #  okay areas 1.6/1.09
    'SCHWARZBACH':('SCHWARZBACHFIRN','schwarzbachfirn','A51E/08',True),  #  okay (not area in FoG)
    'SCHWARZBERG':('Schwarzberggletscher* (Teilgl. von B52/24)','schwarzbergg','B52/63n',True),  #  ok area 5.17/5.33
    'SESVENNA':('Sesvenna Vadret da-E (Teilgl. von E03/04)','sesvenna v da-e','E03/11n',True),  #  ok area 0.67/0.33
    'SEEWIJNEN':('SEEWJINEN GLETSCHER','seewjinen g','B52/22',True)  # ok no area in FoG
    'SEX ROUGE':('SEX ROUGE GLACIER DU (Nr. 47)','sex rouge g du','B16/01',True), # ok even though area 0.72/0.27
    'SILLERN':('SILLERE GLETSCHER','sillere g','A55B/11',True),  #  ok (no area in FoG)
    'SILVRETTA':('SILVRETTAGLETSCHER (Nr. 90)','silvrettag','A10G/05',True),  # ok areas 2.74/2.67
    'SIRWOLTE':('','','',False),  #unclear: might be northerly polygon of Griessernuhorn-N* (c03/04)
    'STEIN':('STEINGLETSCHER (Nr. 53)','steing','B47/01',True),  # ok even though area in Mauro's DB (5.68) slightly bigger than in FoG (5.6)
    'STEINLIMMI':('STEINLIMIGLETSCHER (Nr. 54)','steinlimig','A54E/13',True),  # ok areas 2.21/1.59
    'SULZ':('','','',False),  #  must be HINTERSULZFIRN (Nr. 79), A50I/02,  (lat/lon), but area is bigger (0.26) than in FoG (0.2)
    'SURETTA':('','','',False), # must be SURETTAGLETSCHER-E (Piz Por*), SURETTAGLETSCHER-W (Hauptgl., Nr. 87) and maybe also Suretta Lückli*. FoG point unclear
    'TAELLIBODEN':('','','',False) # no longer digitized in Mauro's inventory => seems to be totally disappeared on orthophoto
    'TIATSCHA':('TIATSCHA VADRET (La Cudera, Nr. 96)','tiatscha v','E50/07',True), # okay areas 2.11/1.82
    'TIEFEN':('TIEFENGLETSCHER (Nr. 66)','tiefeng','A51E/37', True), # ok even though area 3.17/1.99
    'TOURNELON BLANCE':('','','',False),  # should be all (no FoG area): Tournelon Blanc-SE*, Tournelon Blanc-E*, Tournelon Blanc-NE* and Col du Tournelon Blanc*
    'TRIENT':('TRIENT GLACIER DU (Nr. 43)','trient g du','B90/02',True),  # ok area 6.58/5.82
    'TRIEST GL.':('DRIESTGLETSCHER','driestg','B36/17',True),  # ok no area in FoG
    'TRIFT (GADMEN)':('TRIFTGLETSCHER (Nr. 55)','triftg','A54E/24',True),  # ok area 15.33/14.9
    'TSANFLEURON':('TSANFLEURON GLACIER DE (Nr. 33)','tsanfleuron g de','B22/01',True),  # ok area 3.78/2.64
    'TSCHIERVA':('','','',False),  # probably both TSCHIERVA VADRETTIN DA (0.41) and TSCHIERVA VADRET DA (Nr. 93) (5.09), FoG area is 6.83
    'TSCHINGEL':('','','',False),  # is TSCHINGELFIRN (Nr. 60) (A54/M21),  and maybe also Tschingelspitz-S* (A54M/51n) and Tschingelgrat-S* (A54M/52n)
    'TSIDJIORE NOUVE':('TSIJIORE NOUVE GLACIER DE (Nr. 28)','tsijiore nouve g de','B73/16',True), # ok even though area 3.12/2.72
    'TURTMANN (WEST)':('','','',False),  # unclear: TURTMANNGLETSCHER (Teilgl. von B60/09) is only about half (5.5) of turtmann-w in FoG (11km2)
    'UNT.GRINDELWALD':('','','',False),  # might be OBERS ISCHMEER (Teilgl. von A54L/19)
    'UNTERAAR':('UNTERAARGLETSCHER (Teilgl.von A54G/50n) Nr. 51','unteraarg','A54G/11',True), # ok area 22.7/22.5
    'VAL TORTA':('VALTORTA VADRET','valtorta v','E46/06',True),  # ok area 0.17/0.06
    'VALLEGGIA':('VALLEGGIA GH. DI (Nr. 117)','valleggia gh. di','C33/08',True),  # ok area 0.59/0.30
    'VALSOREY':('Valsorey (Teilgl. von B84/15)*','valsorey','B84/27n',True),  #  ok area 2.34/1.9
    'VERSTANKLA':('Verstanclagletscher (Teilgl. von A10G/08)','verstanclag','A10G/24n',True), #ok area 1.06/0.71
    'VORAB':('VORAB GLATSCHER DIL (Nr. 85)','vorab glatscher dil','A14P/01',True), # ok area 2.51/1.22, could also be Vorabsattel-W* (but drain ins other valley)
    'WALLENBUR':('WALLENBURFIRN (Nr. 71)','wallenburf','A51F/24',True), # ok area 1.7/1.41
    'WANNENHORN GL.':('','','',False),  # should be both WANNENHORNGLETSCHER-NW (Teilgl. von B36/57n) and WANNENHORNGLETSCHER-SE (Teilgl. von B36/57n) together...check PARENT ID!!!
    'WANNENHORN GL. N':('WANNENHORNGLETSCHER-NW (Teilgl. von B36/57n)','wannenhorng-nw','B36/32',True), # must be this one due to FoG lat/lon
    'WANNENHORN GL. S':('WANNENHORNGLETSCHER-SE (Teilgl. von B36/57n)','wannenhorng-se','B36/33',True), # must be this one due to FoG lat/lon
    'WITENWASSEREN':('WITENWASSERENGLETSCHER','witenwassereng','',False), # no FoG area given....could also include Witenwasseren-W*(but drains in another valley)
    'ZENBAECHEN GL.':('ZENBAECHENGLETSCHER','zenbaecheng','B36/18',True), # no FoG area given
    'ZINAL':('ZINAL GLACIER DE (Nr. 22)','zinal g de','B63/05',True),  # ok area 16/13.3 debris?
    'ZMUTT':('ZMUTTGLETSCHER (Nr. 15)','zmuttg','B57/05',True),  # ok area 17.4/13.7 debris?
    
}



In [302]:
print(list(set(pda.NAME.values)-set(links.keys())))

['FLUCHTHORN GL.', 'TAELLIBODEN', 'ROTFIRN NORD', 'SEEWJINEN', 'BELLA TOLA', 'BRESCIANA', 'CROSLINA', 'HINDRE SCHMADRI', 'FINDELEN', 'OBERALETSCH', 'PLATTALVA', 'VORDRE SCHMADRI', 'TSEUDET', 'MONT DURAND', 'GRIESS(KLAUSEN)', 'LENTA', 'GRAND DESERT']


# Begin to establish new dataset containing the columns from the CHANGE file

In [286]:
take_over_id = {}
new_id = {}
for key,value in links.items():
    if links[key][-1] == True:
        take_over_id[key] = value
    else:
        new_id[key] = value
        
print(len(new_id))
print(len(take_over_id))

36
106


In [ ]:
entries_new_pdd = pd.DataFrame(columns=pdd.columns.values)
entries_new_pda = pd.DataFrame(columns=pda.columns.values)

In [ ]:
for idx,cols in pdf.iterrows():
    # here comes Horst's name test: if the glacier is already present, use its ID, leave the GENERAL file as is and fill
    # in the MB data only
    # else: create new entry in GENERAL file and in CHANGE file
    
    # if   :
    
    # else:
    
    # create new ID
    gid = new_id_range[idx]
    
    # set the REMARKS (added at the end)
    REMARKS_pdd = ''
    REMARKS_pda = ''
    
    # two new DFs for the different FoG files
    glacier_pdd = pd.DataFrame([['']*len(pdd.columns.values)], columns=pdd.columns.values)
    glacier_pda = pd.DataFrame([['']*len(pda.columns.values)], columns=pda.columns.values)
    
    glacier_pdd.POLITICAL_UNIT = 'CH'
    glacier_pdd.NAME = cols.Glacier_name_SGI2010.upper()
    glacier_pdd.WGMS_ID = gid
    glacier_pdd.YEAR = cols.t2_year
    glacier_pdd.LOWER_BOUND = 9999
    glacier_pdd.UPPER_BOUND = 9999
    glacier_pdd.AREA_SURVEY_YEAR = cols.area_t2_km2
    glacier_pdd.AREA_CHANGE = (cols.area_t2_km2 - cols.area_t1_km2) * 1000  # *1000: unit difference (1000m2 and km2)
    glacier_pdd.AREA_CHANGE_UNC = np.nan
    glacier_pdd.THICKNESS_CHG = np.nan                  # can be calculated???
    glacier_pdd.THICKNESS_CHG_UNC = np.nan
    glacier_pdd.VOLUME_CHANGE = cols.dvol_mio_m3_between_t1_and_t2 *1000   # *1000: unit difference (1000m3 / 1000000m3)
    glacier_pdd.VOLUME_CHANGE_UNC = cols.uncertainty_dvol_between_t1_and_t2_mio_m3 # *1000:unit difference (1000m3 / 1000000m3)
    glacier_pdd.SURVEY_DATE = int(cols.t2_year *10000 + 9999)  # in order to fulfill the requirements (month/day unknown)
    glacier_pdd.SD_PLATFORM_METHOD =  'aC'   # must be determined
    REMARKS_pdd = REMARKS_pdd + 'Survey date method: PL.'
    glacier_pdd.REFERENCE_DATE = int(cols.t1_year *10000 + 9999)  # in order to fulfill the requirements (month/day unknown)
    glacier_pdd.RD_PLATFORM_METHOD = 'aC'   # must be determined
    REMARKS_pdd = REMARKS_pdd + ' Reference date method: PM.'
    glacier_pdd.PUB_IN_FOG = int(2016)
    glacier_pdd.INVESTIGATOR = 'Mauro Fischer, Matthias Huss, Martin Hoelzle'
    glacier_pdd.SPONS_AGENCY = 'Department of Geosciences, University of Fribourg, 1700 Fribourg, Switzerland'
    glacier_pdd.REFERENCE = 'Fischer et al. (2015); The Cryosphere, 9, 2016'
    REMARKS_pdd = REMARKS_pdd + ' ID SGI 1973: %s.' % cols.ID_SGI1973
    REMARKS_pdd = REMARKS_pdd + ' ID SGI 2010: %s.' % cols.Code_SGI2010
    # at the very end 
    glacier_pdd.REMARKS = REMARKS_pdd
    
    
    glacier_pda.POLTITICAL_UNIT = 'CH'
    glacier_pda.NAME = cols.Glacier_name_SGI2010.upper()
    glacier_pda.WGMS_ID = int(gid) 
    glacier_pda.LATITUDE = pdfll[pdfll.Glacier_name_SGI2010 == cols.Code_SGI2010]['y WGS84)'].values[0]
    glacier_pda.LONGITUDE = pdfll[pdfll.Glacier_name_SGI2010 == cols.Code_SGI2010]['Location(x WGS84)'].values[0]
    glacier_pda['REGION'] = 'Central Europe'
    glacier_pda['SUBREGION'] = 'Alps'

    
    entries_new_pdd = entries_new_pdd.append(glacier_pdd, ignore_index=True)
    entries_new_pda = entries_new_pda.append(glacier_pda, ignore_index=True)

In [ ]:
entries_new_pdd[['WGMS_ID', 'YEAR', 'LOWER_BOUND', 'UPPER_BOUND', 'SURVEY_DATE', 'REFERENCE_DATE', 'PUB_IN_FOG']] = entries_new_pdd[['WGMS_ID', 'YEAR', 'LOWER_BOUND', 'UPPER_BOUND', 'SURVEY_DATE', 'REFERENCE_DATE', 'PUB_IN_FOG']].astype(int)
entries_new_pdd.head()

In [ ]:
len(pdf)

In [ ]:
entries_new_pda.head()
#pdfll[pdfll.Glacier_name_SGI2010 == cols.Code_SGI2010]['y WGS84)'].values[0]

In [ ]:
pdfll.head()

In [ ]:
cols.Code_SGI2010

In [ ]:
pdfll[pdfll.Glacier_name_SGI2010 == cols.Code_SGI2010]